In [14]:
import random
import string

import redis

In [3]:
r = redis.Redis(host='localhost', port=6379)

In [52]:
variant = "chr10:10403244:A:C"
def get_random_string(length):
    return ''.join(random.choices(string.ascii_letters, k=length))
    
get_random_string(length=len(variant))

'WqhguhPFoycMOZILzn'

In [46]:
import pandas as pd

df_pangolin = pd.read_csv("brca.pangolin.csv")
df_pangolin.head()

,gene,CHROM,POS,REF,ALT,Pangolin
0,BRCA1,17,41276135,T,G,ENSG00000012048.23_1|-46:0.0|-45:0.0|Warnings:
1,BRCA1,17,41276135,T,C,ENSG00000012048.23_1|-1:0.8600000143051147|-3:...
2,BRCA1,17,41276135,T,A,ENSG00000012048.23_1|-46:0.0|-45:0.0|Warnings:
3,BRCA1,17,41276134,T,G,ENSG00000012048.23_1|-15:0.8100000023841858|-2...
4,BRCA1,17,41276134,T,C,ENSG00000012048.23_1|-1:0.8600000143051147|-2:...


In [39]:
def get_min_avg_len(df, column):
    avg_len = df[column].str.len().mean()
    avg_len = int(round(avg_len, 0))
    max_len = df[column].str.len().max()
    min_len = df[column].str.len().min()
    return  min_len, avg_len, max_len

In [48]:
pangolin_min, pangolin_avg, pangolin_max = get_min_avg_len(df_pangolin, "Pangolin")
print(f"pangolin: {pangolin_min}, {pangolin_avg}, {pangolin_max}")

pangolin: 44, 68, 78


In [49]:
df_spip = pd.read_csv("spip_results.txt", sep="\t")
df_spip.head()

,gene,varID,Interpretation,InterConfident,SPiPscore,strand,gNomen,varType,ntChange,ExonInfo,...,nearestPosSStoCrypt,nearestDistSStoCrypt,posCryptWT,probaCryptWT,classProbaCryptWT,posSSPhysio,probaSSPhysio,classProbaSSPhysio,probaSSPhysioMut,classProbaSSPhysioMut
0,NF1,NM_000267:c.1000del,NTR,05.1 % [02.47 % - 09.18 %],0.014,+,29527551_29527551,del,del,Exon 9,...,29527613,-51,29527563,0.000023,No,29527439,0.005161,No,0.005161,No
1,NF1,NM_000267:c.1007_1008delinsAA,NTR,04.35 % [02.01 % - 08.09 %],0.008,+,29527558_29527559,delins,delinsAA,Exon 9,...,29527613,-54,29527563,0.000023,No,29527439,0.005161,No,0.005161,No
2,NF1,NM_000267:c.1007G>A,Alter ESR,98.54 % [94.83 % - 99.82 %],0.862,+,29527558,substitution,G>A,Exon 9,...,29527613,-55,29527563,0.000023,No,29527439,0.005161,No,0.005161,No
3,NF1,NM_000267:c.1008G>A,NTR,04.35 % [02.01 % - 08.09 %],0.012,+,29527559,substitution,G>A,Exon 9,...,29527613,-54,29527563,0.000023,No,29527439,0.005161,No,0.005161,No
4,NF1,NM_000267:c.1016del,NTR,04.35 % [02.01 % - 08.09 %],0.020,+,29527567_29527567,del,del,Exon 9,...,29527613,-51,29527563,0.000023,No,29527439,0.005161,No,0.005161,No


In [50]:
spip_intp_min, spip_intp_avg, spip_intp_max = get_min_avg_len(df_spip, "Interpretation")
print(f"spip Interpretation: {get_min_avg_len(df_spip, 'Interpretation')}")

spip Interpretation: (3, 9, 50)


In [51]:
spip_spice_min, spip_spice_avg, spip_spice_max = get_min_avg_len(df_spip, "SPiCEinter_2thr")
print(f"spip SPiCEinter_2thr: {get_min_avg_len(df_spip, 'SPiCEinter_2thr')}")

spip SPiCEinter_2thr: (3, 20, 28)


#### Redis Hashes

In [53]:
variant = get_random_string(len(variant))
pang = get_random_string(pangolin_avg)
spip_intp = get_random_string(spip_intp_avg)
spip_spice = get_random_string(spip_spice_avg)

r.hset(
    variant,
    mapping={
        "pang": pang,
        "spip_int": spip_intp,
        "spip_spice": spip_spice,
    },
)

r.memory_usage(variant)

384

#### Redis Strings

In [54]:
key_pang = get_random_string(len(variant)+len(":pang"))
value_pang = get_random_string(pangolin_avg)
r.set(key_pang, value_pang)
mem_pangolin = r.memory_usage(key_pang)

key_spip = get_random_string(len(variant)+len(":spip"))
sample_spip_value = f"{spip_intp}\t{spip_spice}"
value_spip = get_random_string(len(sample_spip_value))
r.set(key_spip, value_spip)
mem_spip = r.memory_usage(key_spip)

mem_pangolin_spip = mem_pangolin + mem_spip
print(f"pangolin: {mem_pangolin}, spip: {mem_spip}, pangolin+spip: {mem_pangolin_spip}")

pangolin: 152, spip: 112, pangolin+spip: 264


In [58]:
value_pang

'yaRfzBhBibDDnipvRgReMlquRzPkfZLwepqnxZCXOBIHCVQoVpbDjmiwcBonfOYwOisE'

In [56]:
CACHED_VARIANTS = 36_000_000_000

storage_GiB = CACHED_VARIANTS * mem_pangolin_spip / 1024 / 1024 / 1024 
print(f"Storage: {storage_GiB} GiB")

Storage: 8851.289749145508 GiB


In [57]:
amount_on_10GB = 10 * 1024 * 1024 * 1024 / mem_pangolin_spip
print(f"Amount of variants on 10GB: {amount_on_10GB}")

Amount of variants on 10GB: 40672038.78787879


40 672 038 variants in 10GB

#### save as bytes

In [62]:
key_pang = get_random_string(len(variant)+len(":pang"))
value_pang = get_random_string(pangolin_avg)
r.set(bytes(key_pang, "utf-8"), bytes(value_pang, "utf-8"))
mem_pangolin = r.memory_usage(key_pang)

key_spip = get_random_string(len(variant)+len(":spip"))
sample_spip_value = f"{spip_intp}\t{spip_spice}"
value_spip = get_random_string(len(sample_spip_value))
r.set(bytes(key_spip, "utf-8"), bytes(value_spip, "utf-8"))
mem_spip = r.memory_usage(key_spip)

mem_pangolin_spip = mem_pangolin + mem_spip
print(f"pangolin: {mem_pangolin}, spip: {mem_spip}, pangolin+spip: {mem_pangolin_spip}")

pangolin: 152, spip: 112, pangolin+spip: 264


In [63]:
import pickle
# pickle key and value
pickle_key_pang = pickle.dumps(key_pang)
pickle_value_pang = pickle.dumps(value_pang)
r.set(pickle_key_pang, pickle_value_pang)
mem_pangolin = r.memory_usage(pickle_key_pang)

pickle_key_spip = pickle.dumps(key_spip)
pickle_value_spip = pickle.dumps(value_spip)
r.set(pickle_key_spip, pickle_value_spip)
mem_spip = r.memory_usage(pickle_key_spip)

mem_pangolin_spip = mem_pangolin + mem_spip
print(f"pangolin: {mem_pangolin}, spip: {mem_spip}, pangolin+spip: {mem_pangolin_spip}")

pangolin: 184, spip: 144, pangolin+spip: 328


In [64]:
import msgpack
# msgpack key and value
msgpack_key_pang = msgpack.packb(key_pang)
msgpack_value_pang = msgpack.packb(value_pang)
r.set(msgpack_key_pang, msgpack_value_pang)
mem_pangolin = r.memory_usage(msgpack_key_pang)

msgpack_key_spip = msgpack.packb(key_spip)
msgpack_value_spip = msgpack.packb(value_spip)
r.set(msgpack_key_spip, msgpack_value_spip)
mem_spip = r.memory_usage(msgpack_key_spip)

mem_pangolin_spip = mem_pangolin + mem_spip
print(f"pangolin: {mem_pangolin}, spip: {mem_spip}, pangolin+spip: {mem_pangolin_spip}")

pangolin: 152, spip: 112, pangolin+spip: 264


In [67]:
import zlib
 
# zlib key and value
zlib_key_pang = zlib.compress(bytes(key_pang, "utf-8"))
zlib_value_pang = zlib.compress(bytes(value_pang, "utf-8"))
r.set(zlib_key_pang, zlib_value_pang)
mem_pangolin = r.memory_usage(zlib_key_pang)

zlib_key_spip = zlib.compress(bytes(key_spip, "utf-8"))
zlib_value_spip = zlib.compress(bytes(value_spip, "utf-8"))
r.set(zlib_key_spip, zlib_value_spip)
mem_spip = r.memory_usage(zlib_key_spip)

mem_pangolin_spip = mem_pangolin + mem_spip
print(f"pangolin: {mem_pangolin}, spip: {mem_spip}, pangolin+spip: {mem_pangolin_spip}")

pangolin: 160, spip: 128, pangolin+spip: 288
